# Spark in Action - Chapter 4 Scala Version - Lab 200

In [ ]:
import java.util.{Arrays, List}
import org.apache.spark.sql.{Dataset, Encoders, SparkSession}

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.41:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1667062220119)
SparkSession available as 'spark'


import java.util.{Arrays, List}
import org.apache.spark.sql.{Dataset, Encoders, SparkSession}


In [ ]:
var mode:String = "noop"

mode: String = noop


In [ ]:
val t0 = System.currentTimeMillis

// Step 1 - Creates a session on a local maste
val spark = SparkSession
                .builder
                .appName("Analysing Catalyst's behavior")
                .master("local[*]")
                .getOrCreate

val t1 = System.currentTimeMillis
println("1. Creating a session ........... " + (t1 - t0))


22/10/29 18:50:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
1. Creating a session ........... 1061


t0: Long = 1667062227021
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6fc94bae
t1: Long = 1667062228082


In [ ]:
// Step 2 - Reads a CSV file with header, stores it in a dataframe
var df = spark.read
            .format("csv")
            .option("header", "true")
            .load("../net.jgp.books.spark.ch04/data/NCHS_-_Teen_Birth_Rates_for_Age_Group_15-19_in_the_United_States_by_County.csv")

val initalDf = df
val t2 = System.currentTimeMillis
println("2. Loading initial dataset ...... " + (t2 - t1))


2. Loading initial dataset ...... 3933


df: org.apache.spark.sql.DataFrame = [Year: string, State: string ... 7 more fields]
initalDf: org.apache.spark.sql.DataFrame = [Year: string, State: string ... 7 more fields]
t2: Long = 1667062232015


In [ ]:
// Step 3 - Build a bigger dataset
for(_ <-  0.to(60)){
  df = df.union(initalDf)
}
val t3 = System.currentTimeMillis
println("3. Building full dataset ........ " + (t3 - t2))


3. Building full dataset ........ 1020


t3: Long = 1667062233035


In [ ]:
// Step 4 - Cleanup. preparation
df = df.withColumnRenamed("Lower Confidence Limit", "lcl")
       .withColumnRenamed("Upper Confidence Limit", "ucl")

val t4 = System.currentTimeMillis
println("4. Clean-up ..................... " + (t4 - t3))


4. Clean-up ..................... 231


df: org.apache.spark.sql.DataFrame = [Year: string, State: string ... 7 more fields]
t4: Long = 1667062233266


In [ ]:
// Step 5 - Transformation
if (mode.compareToIgnoreCase("noop") != 0) {
  df = df.withColumn("avg", expr("(lcl+ucl)/2"))
         .withColumn("lcl2", col("lcl"))
         .withColumn("ucl2", col("ucl"))
  if (mode.compareToIgnoreCase("full") == 0)
    df = df.drop("avg","lcl2","ucl2")
}

val t5 = System.currentTimeMillis
println("5. Transformations  ............. " + (t5 - t4))


5. Transformations  ............. 251


t5: Long = 1667062233517


In [ ]:
// Step 6 - Action
df.collect
val t6 = System.currentTimeMillis
println("6. Final action ................. " + (t6 - t5))

println("")
println("# of records .................... " + df.count)

22/10/29 18:54:06 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 192.168.1.41:60730 in 10000 milliseconds
22/10/29 18:54:28 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.s

In [ ]:
spark.stop()